## Data extraction

This notebook processes CSV and PDF reports to use as input in the RAG model. 

##### 1. Startup PDFs
- Identifies and extracts company information from PDF using specified pattern (AutoTechInsight and Seedtable specific)
- Saves all extracted information as text file
- Note: Seedtable dataset was cleaned manually after extraction

##### 2. Automotive Tech Patents & Papers CSV
- Converts CSV into readable text file
- No filtering, saves full dataset as text file

##### 3. Technology and Automative PDF reports:
- Extracts text from PDF
- Cleans the text (removes noise, whitespaces, headers)
- Detects major sections using regex patterns (Report specific patterns for WEF, McKInsey and BCG)
- Extracts key sections (summary, conclusions, methodology, etc.)
- Outputs a structured text file containing key sections and full text

Information on data sources:
- Overview of data /01_data/rag_automotive_tech/metadata.json
- Original reports found in 01/data/rag_automotive_tech/raw_data
- Processed documents saved in 01/data/rag_automotive_tech/processed

In [4]:
# Import libraries

from pathlib import Path
import pandas as pd
import csv
import PyPDF2
import re
import os

### Startup PDFs

In [ ]:
# AutoTechInsight

def extract_startups_from_autotechinsight():
    """Extract startup data from autotechinsight_startups_automative.pdf"""
    
    # Define paths
    base_path = "../../01_data/rag_automotive_tech"
    raw_sources_path = os.path.join(base_path, "raw_sources", "startups")
    processed_path = os.path.join(base_path, "processed", "startups")
    
    # Create processed directory if it doesn't exist
    os.makedirs(processed_path, exist_ok=True)
    
    pdf_path = os.path.join(raw_sources_path, "autotechinsight_startups_automative.pdf")
    output_path = os.path.join(processed_path, "autotechinsight_startups_processed.txt")
    
    print(f"📄 Processing AutotechInsight PDF: {pdf_path}")
    
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            full_text = ""
            for page in pdf_reader.pages:
                full_text += page.extract_text() + "\n"
        
        # Debug: Show first 500 characters to understand structure
        print("First 500 chars of PDF:")
        print(full_text[:500])
        print("---")
        
        startups = []
        
        # Split by "STARTUP" pattern - the actual structure uses spaces, not pipes
        sections = re.split(r'STARTUP\s+', full_text)
        
        # Skip the first section (header/content before first startup)
        for section in sections[1:]:
            startup = {}
            
            # Extract location and date - pattern for "Europe 27-Nov-2025"
            location_match = re.search(r'^([A-Za-z\s/]+)\s+(\d{2}-[A-Za-z]+-\d{4})', section)
            if location_match:
                startup['location'] = location_match.group(1).strip()
                startup['date'] = location_match.group(2).strip()
                
                # Remove the location/date part from the section to get clean content
                section = section[location_match.end():].strip()
            
            # Split into lines and clean up
            lines = [line.strip() for line in section.split('\n') if line.strip()]
            
            if lines:
                # First non-empty line after location/date is the startup name
                startup['name'] = lines[0]
                
                # The rest is description
                if len(lines) > 1:
                    startup['description'] = ' '.join(lines[1:])
                else:
                    startup['description'] = "No description available"
            
            if startup.get('name'):
                startups.append(startup)
                print(f"Found: {startup.get('name')} - {startup.get('location', 'N/A')}")
        
        # Save to text file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("AUTOTECHINSIGHT STARTUPS DATABASE\n")
            f.write("=" * 50 + "\n\n")
            
            for idx, startup in enumerate(startups, 1):
                f.write(f"STARTUP #{idx}:\n")
                f.write(f"Name: {startup.get('name', 'N/A')}\n")
                f.write(f"Location: {startup.get('location', 'N/A')}\n")
                f.write(f"Date: {startup.get('date', 'N/A')}\n")
                f.write(f"Description: {startup.get('description', 'N/A')}\n")
                f.write("-" * 40 + "\n\n")
        
        print(f"✅ Extracted {len(startups)} startups from AutotechInsight")
        print(f"📁 File saved: {output_path}")
        return len(startups)
        
    except Exception as e:
        print(f"❌ Error processing AutotechInsight PDF: {e}")
        import traceback
        traceback.print_exc()
        return 0

# Run AutotechInsight extraction
if __name__ == "__main__":
    extract_startups_from_autotechinsight()

📄 Processing AutotechInsight PDF: ../../01_data/rag_automotive_tech/raw_sources/startups/autotechinsight_startups_automative.pdf
First 500 chars of PDF:
  
STARTUP Europe 27-Nov-2025
2hire
2Hire develops software that provides mobility solutions by turning all kinds of vehicles into
connected cars. It offers 2hire board and a dongle which helps in interaction with the vehicle by
application. It equips vehicles with plug-and-play devices, smartphones, real-time data…
STARTUP North America 27-Nov-2025
Actnano
actnano, Inc., a revolutionary surface protection technology company, engages in the research,
design, development, manufacture, and distrib
---
Found: 2hire - Europe
Found: Actnano - North America
Found: Actronika - Europe
Found: AD Knight - Middle East/Africa
Found: Advano - North America
Found: Affectiva - North America
Found: Aidriving - Greater China
Found: AImotive - Europe
Found: Ainstein - North America
Found: Alchemy - North America
Found: AmpUp - North America
Found: Appli

In [ ]:
# Seedtable

def extract_startups_from_seedtable():
    """Extract startup data from seedtable_automative_startups_in_2025.pdf"""
    
    # Define paths
    base_path = "../../01_data/rag_automotive_tech"
    raw_sources_path = os.path.join(base_path, "raw_sources", "startups")
    processed_path = os.path.join(base_path, "processed", "startups")
    
    # Create processed directory if it doesn't exist
    os.makedirs(processed_path, exist_ok=True)
    
    pdf_path = os.path.join(raw_sources_path, "seedtable_automative_startups_in_2025.pdf")
    output_path = os.path.join(processed_path, "seedtable_startups_processed.txt")
    
    print(f"📊 Processing Seedtable PDF: {pdf_path}")
    
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            full_text = ""
            for page in pdf_reader.pages:
                full_text += page.extract_text() + "\n"
        
        print("🔍 Analyzing PDF structure...")
        
        # Use a more targeted approach based on the actual structure
        startups = []
        
        # Split by clear section markers - look for patterns like "Funding Rounds" and "Money raised"
        sections = re.split(r'(\d+)\s*Funding Rounds', full_text)
        
        # The sections will be: [pre_content, funding_count1, content1, funding_count2, content2, ...]
        for i in range(1, len(sections), 2):
            if i + 1 < len(sections):
                funding_rounds = sections[i]
                content_section = sections[i + 1]
                
                startup = {'funding_rounds': funding_rounds}
                
                # Extract money raised from this section
                money_match = re.search(r'\$([\d\.,]+[BMK]?)\s*Money raised', content_section)
                if money_match:
                    startup['money_raised'] = f"${money_match.group(1)}"
                
                # Extract name - look backward in previous content
                if i > 1:
                    previous_content = sections[i - 1]
                    # Look for the last prominent line before the funding rounds
                    previous_lines = [line.strip() for line in previous_content.split('\n') if line.strip()]
                    for line in reversed(previous_lines[-3:]):  # Check last 3 lines
                        if (len(line) > 2 and 
                            not line.startswith('$') and 
                            not 'Funding' in line and
                            not 'Money' in line and
                            not 'Location:' in line and
                            not 'Industries:' in line and
                            not 'Key people:' in line and
                            not '@' in line and
                            not re.search(r'^\d', line) and
                            line not in ['What3words', 'LUMINAR', 'PolyAI', 'Helm.ai', 'Metawave', 'ZeroAvia', 'GoMechanic', 'Northvolt']):
                            startup['name'] = line
                            break
                
                # Extract description - text between funding rounds and money raised
                # Remove money raised part first to isolate description
                content_without_money = re.sub(r'\$[\d\.,]+[BMK]?\s*Money raised', '', content_section)
                # Get text until Industries or Location
                desc_match = re.search(r'^(.+?)(?=Industries:|Location:|Key people:|$)', content_without_money, re.DOTALL)
                if desc_match:
                    description = desc_match.group(1).strip()
                    # Clean up - remove extra whitespace
                    description = re.sub(r'\s+', ' ', description)
                    if len(description) > 10:  # Only use if substantial
                        startup['description'] = description
                
                # Extract industries
                industries_match = re.search(r'Industries:\s*(.+?)(?=Location:|Key people:|$)', content_section, re.DOTALL)
                if industries_match:
                    industries = industries_match.group(1).strip()
                    industries = re.sub(r'\s+', ' ', industries)
                    startup['industries'] = industries
                
                # Extract location
                location_match = re.search(r'Location:\s*(.+?)(?=Industries:|Key people:|$)', content_section, re.DOTALL)
                if location_match:
                    location = location_match.group(1).strip()
                    location = re.sub(r'\s+', ' ', location)
                    startup['location'] = location
                
                # Only add if we have at least a name and funding info
                if startup.get('name') and startup.get('funding_rounds'):
                    startups.append(startup)
                    print(f"✅ Found: {startup.get('name')} | Funding: {startup.get('funding_rounds')}")
        
        # If we didn't find enough, try alternative method
        if len(startups) < 10:
            print("Trying alternative extraction method...")
            startups = extract_using_money_patterns(full_text)
        
        # Save to text file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("SEEDTABLE AUTOMOTIVE STARTUPS 2025\n")
            f.write("=" * 50 + "\n\n")
            
            for idx, startup in enumerate(startups, 1):
                f.write(f"STARTUP #{idx}:\n")
                f.write(f"Name: {startup.get('name', 'N/A')}\n")
                f.write(f"Funding Rounds: {startup.get('funding_rounds', 'N/A')}\n")
                f.write(f"Money Raised: {startup.get('money_raised', 'N/A')}\n")
                f.write(f"Description: {startup.get('description', 'N/A')}\n")
                f.write(f"Industries: {startup.get('industries', 'N/A')}\n")
                f.write(f"Location: {startup.get('location', 'N/A')}\n")
                f.write("-" * 50 + "\n\n")
        
        print(f"🎉 Successfully extracted {len(startups)} startups from Seedtable")
        print(f"📁 File saved: {output_path}")
        return len(startups)
        
    except Exception as e:
        print(f"❌ Error processing Seedtable PDF: {e}")
        import traceback
        traceback.print_exc()
        return 0

def extract_using_money_patterns(full_text):
    """Alternative extraction using money patterns"""
    startups = []
    
    # Split by money raised patterns
    money_sections = re.split(r'\$([\d\.,]+[BMK]?)\s*Money raised', full_text)
    
    for i in range(1, len(money_sections), 2):
        if i + 1 < len(money_sections):
            money_amount = money_sections[i]
            content_after_money = money_sections[i + 1]
            content_before_money = money_sections[i - 1] if i > 0 else ""
            
            startup = {'money_raised': f"${money_amount}"}
            
            # Look for funding rounds in the content before money
            funding_match = re.search(r'(\d+)\s*Funding Rounds', content_before_money)
            if funding_match:
                startup['funding_rounds'] = funding_match.group(1)
            
            # Extract name from before the funding rounds
            if funding_match:
                # Get the text before funding rounds
                text_before_funding = content_before_money[:funding_match.start()]
                lines = [line.strip() for line in text_before_funding.split('\n') if line.strip()]
                for line in reversed(lines[-2:]):  # Check last 2 lines
                    if (len(line) > 2 and not any(keyword in line for keyword in 
                        ['Funding', 'Money', 'Location:', 'Industries:', 'Key people:', '$'])):
                        startup['name'] = line
                        break
            
            # Extract description - text between funding and money
            if funding_match and startup.get('name'):
                desc_start = funding_match.end()
                desc_end = content_before_money.find('$') if '$' in content_before_money else len(content_before_money)
                description = content_before_money[desc_start:desc_end].strip()
                if description and len(description) > 10:
                    startup['description'] = re.sub(r'\s+', ' ', description)
            
            # Extract industries and location from after money
            industries_match = re.search(r'Industries:\s*(.+?)(?=Location:|Key people:|$)', content_after_money, re.DOTALL)
            if industries_match:
                startup['industries'] = re.sub(r'\s+', ' ', industries_match.group(1).strip())
            
            location_match = re.search(r'Location:\s*(.+?)(?=Industries:|Key people:|$)', content_after_money, re.DOTALL)
            if location_match:
                startup['location'] = re.sub(r'\s+', ' ', location_match.group(1).strip())
            
            if startup.get('name') and startup.get('funding_rounds'):
                startups.append(startup)
                print(f"✅ Found via money: {startup.get('name')}")
    
    return startups

# Run Seedtable extraction
if __name__ == "__main__":
    extract_startups_from_seedtable()

📊 Processing Seedtable PDF: ../../01_data/rag_automotive_tech/raw_sources/startups/seedtable_automative_startups_in_2025.pdf
🔍 Analyzing PDF structure...
✅ Found: https://www.seedtable.com/best-automotive-industry-startups?utm_source=chatgpt.com 1/35 | Funding: 9
✅ Found: Mobius Motors | Funding: 3
✅ Found: Joel Jackson (entrepreneur) | Funding: 4
✅ Found: Helm.ai (company) | Funding: 3
✅ Found: Metawave Corporation | Funding: 3
✅ Found: https://www.seedtable.com/best-automotive-industry-startups?utm_source=chatgpt.com 4/35 | Funding: 3
✅ Found: London | Funding: 5
✅ Found: Amit Bhasin | Funding: 12
✅ Found: Horizon Robotics | Funding: 10
✅ Found: Pony.ai | Funding: 9
✅ Found: Getaround | Funding: 5
✅ Found: WayRay | Funding: 3
✅ Found: CYCY Vision | Funding: 2
✅ Found: SpinCar | Funding: 4
✅ Found: https://www.seedtable.com/best-automotive-industry-startups?utm_source=chatgpt.com 8/35 | Funding: 2
✅ Found: Gogoro | Funding: 1
✅ Found: VeriTX Corp. | Funding: 4
✅ Found: VoyaVoyage Auto

### Automotive Tech Patents & Papers CSV

In [5]:
# Automotive Papers CSV

def process_automotive_papers(csv_path, output_path):
    """Process automotive research papers CSV to text file"""
    
    csv_file = Path(csv_path)
    if not csv_file.exists():
        print(f"❌ ERROR: CSV file not found at: {csv_file.absolute()}")
        return False
    
    print(f"✅ Found automotive papers CSV: {csv_file}")
    
    try:
        # Read the entire CSV
        df = pd.read_csv(csv_path)
        
        print(f"📊 Total papers loaded: {len(df)}")
        print(f"📋 Columns available: {df.columns.tolist()}")
        
        # Create output directory
        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Save papers to .txt file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("AUTOMOTIVE RESEARCH PAPERS DATASET\n")
            f.write(f"Total papers: {len(df)}\n")
            f.write(f"Source: {csv_path}\n")
            f.write("="*60 + "\n\n")
            
            for i, (_, paper) in enumerate(df.iterrows(), 1):
                f.write(f"RESEARCH PAPER #{i}:\n")
                
                # Extract fields based on actual column names
                title = paper.get('title', 'N/A')
                year = paper.get('year_published', 'N/A')
                authors = paper.get('authors', 'N/A')
                abstract = paper.get('abstract', 'N/A')
                
                f.write(f"  Title: {title}\n")
                f.write(f"  Year Published: {year}\n")
                f.write(f"  Authors: {authors}\n")
                f.write(f"  Abstract: {abstract}\n")
                f.write("-" * 50 + "\n\n")
        
        print(f"✅ Successfully saved {len(df)} automotive papers to: {output_path}")
        return True
        
    except Exception as e:
        print(f"❌ Error processing automotive papers: {e}")
        return False

# Process Automotive Tech Patents CSV
def process_automotive_patents(csv_path, output_path):
    """Process automotive technology patents CSV to text file"""
    
    csv_file = Path(csv_path)
    if not csv_file.exists():
        print(f"❌ ERROR: CSV file not found at: {csv_file.absolute()}")
        return False
    
    print(f"✅ Found automotive patents CSV: {csv_file}")
    
    try:
        # Read the entire CSV
        df = pd.read_csv(csv_path)
        
        print(f"📊 Total patents loaded: {len(df)}")
        print(f"📋 Columns available: {df.columns.tolist()}")
        
        # Create output directory
        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Save patents to .txt file
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("AUTOMOTIVE TECHNOLOGY PATENTS DATASET\n")
            f.write(f"Total patents: {len(df)}\n")
            f.write(f"Source: {csv_path}\n")
            f.write("="*60 + "\n\n")
            
            for i, (_, patent) in enumerate(df.iterrows(), 1):
                f.write(f"PATENT #{i}:\n")
                
                # Extract fields based on actual column names
                lens_id = patent.get('lens_id', 'N/A')
                jurisdiction = patent.get('jurisdiction', 'N/A')
                kind = patent.get('kind', 'N/A')
                date_published = patent.get('date_published', 'N/A')
                abstract = patent.get('abstract', 'N/A')
                
                f.write(f"  Lens ID: {lens_id}\n")
                f.write(f"  Jurisdiction: {jurisdiction}\n")
                f.write(f"  Kind: {kind}\n")
                f.write(f"  Date Published: {date_published}\n")
                f.write(f"  Abstract: {abstract}\n")
                f.write("-" * 50 + "\n\n")
        
        print(f"✅ Successfully saved {len(df)} automotive patents to: {output_path}")
        return True
        
    except Exception as e:
        print(f"❌ Error processing automotive patents: {e}")
        return False

# Process Automotive Papers
print("🚗 PROCESSING AUTOMOTIVE RESEARCH PAPERS...")
papers_csv_path = "../../01_data/rag_automotive_tech/raw_sources/automotive_papers/lens_scholarly_ai_automotive_2025.csv"
papers_output_path = "../../01_data/rag_automotive_tech/processed/automotive_papers/automotive_papers_processed.txt"

papers_success = process_automotive_papers(papers_csv_path, papers_output_path)

# Process Automotive Patents
print("\n📜 PROCESSING AUTOMOTIVE TECHNOLOGY PATENTS...")
patents_csv_path = "../../01_data/rag_automotive_tech/raw_sources/automotive_tech_patents/lens_patent_ai_automotive_2025.csv"
patents_output_path = "../../01_data/rag_automotive_tech/processed/automotive_tech_patents/automotive_patents_processed.txt"

patents_success = process_automotive_patents(patents_csv_path, patents_output_path)

# Summary
print("\n" + "="*60)
print("📊 PROCESSING SUMMARY:")
print(f"✅ Automotive Papers: {'SUCCESS' if papers_success else 'FAILED'}")
print(f"✅ Automotive Patents: {'SUCCESS' if patents_success else 'FAILED'}")

if success and papers_success and patents_success:
    print("\n🎉 All datasets processed successfully!")
    print("📁 Processed files:")
    print(f"   - Papers: {papers_output_path}") 
    print(f"   - Patents: {patents_output_path}")
else:
    print("\n💥 Some processing failed! Check errors above.")

🚗 PROCESSING AUTOMOTIVE RESEARCH PAPERS...
❌ ERROR: CSV file not found at: /Users/siriamandaraaf/Documents/spiced-academy/04-course-material/spiced-academy-data-science/capstone-project/innovation-intelligence-suite/03_notebooks/rag_notebooks/../../01_data/rag_automotive_tech/raw_sources/automotive_papers/lens_scholarly_ai_automotive_2025.csv

📜 PROCESSING AUTOMOTIVE TECHNOLOGY PATENTS...
❌ ERROR: CSV file not found at: /Users/siriamandaraaf/Documents/spiced-academy/04-course-material/spiced-academy-data-science/capstone-project/innovation-intelligence-suite/03_notebooks/rag_notebooks/../../01_data/rag_automotive_tech/raw_sources/automotive_tech_patents/lens_patent_ai_automotive_2025.csv

📊 PROCESSING SUMMARY:
✅ Automotive Papers: FAILED
✅ Automotive Patents: FAILED


NameError: name 'success' is not defined

### Technology Reports PDFs

In [5]:
# Extract text from PDFs

def extract_pdf_text(pdf_path):
    """Extract text from PDF file"""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
            return text
    except Exception as e:
        print(f"❌ Error reading PDF {pdf_path}: {e}")
        return ""

# Clean text

def clean_report_text(text):
    """Clean and format report text"""
    # Remove excessive whitespace
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    
    # Remove page numbers and headers
    text = re.sub(r'Page \d+ of \d+', '', text)
    text = re.sub(r'\n\d+\n', '\n', text)
    
    return text.strip()

# Define universal patterns that work across most professional reports
universal_patterns = {
    'executive_summary': [
        r'executive summary',
        r'key insights', 
        r'in brief',
        r'highlights',
        r'overview'
    ],
    'introduction': [
        r'introduction',
        r'foreword',
        r'preface',
        r'background',
        r'context'
    ],
    'methodology': [
        r'methodology',
        r'approach',
        r'research design',
        r'data sources',
        r'analytical framework'
    ],
    'conclusions': [
        r'conclusion',
        r'summary',
        r'key takeaways',
        r'implications',
        r'closing remarks'
    ],
    'references': [
        r'references',
        r'endnotes',
        r'citations',
        r'bibliography',
        r'sources'
    ],
    'appendix': [
        r'appendix',
        r'supplemental',
        r'additional data',
        r'supporting information'
    ]
}

# WEF-Specific Patterns (Technology & Foresight Focus)
wef_specific_patterns = {
    'technology_catalog': [
        r'\d{2}\s+[\w\s]+',  # Matches "01 Structural battery composites"
        r'technolog(y|ies)',
        r'emerging technologies',
        r'top \d+',
        r'structural battery composites',
        r'osmotic power systems',
        r'advanced nuclear technologies',
        r'engineered living therapeutics',
        r'GLP[-\s]?1s',
        r'autonomous biochemical sensing',
        r'green nitrogen fixation',
        r'nanozymes',
        r'collaborative sensing',
        r'generative watermarking'
    ],
    'strategic_outlook': [
        r'strategic outlook',
        r'future implications',
        r'transformative potential',
        r'societal transformation',
        r'strategic implications',
        r'forward-looking'
    ],
    'ecosystem_readiness': [
        r'ecosystem readiness',
        r'readiness map',
        r'stee?p analysis',
        r'social.*technological.*environmental.*economic.*policy',
        r'key actions to achieve scale',
        r'readiness assessment'
    ],
    'foresight_framework': [
        r'strategic foresight',
        r'future trends',
        r'megatrends',
        r'weak signals',
        r'future scenarios',
        r'from weak signals to'
    ],
    'technology_definition': [
        r'what are',
        r'definition of',
        r'overview of',
        r'technical background'
    ]
}

# McKinsey-Specific Patterns (Business & Consulting Focus)
mckinsey_specific_patterns = {
    'business_case': [
        r'business case',
        r'value proposition',
        r'roi',
        r'economic impact',
        r'financial benefits',
        r'cost.*benefit',
        r'value creation',
        r'investment case'
    ],
    'market_analysis': [
        r'market analysis',
        r'industry outlook',
        r'market size',
        r'growth projections',
        r'competitive landscape',
        r'market dynamics',
        r'industry trends'
    ],
    'case_studies': [
        r'case stud(y|ies)',
        r'client examples',
        r'implementation examples',
        r'success stories',
        r'use cases',
        r'client experience',
        r'example.*company'
    ],
    'implementation_roadmap': [
        r'implementation',
        r'roadmap',
        r'action plan',
        r'next steps',
        r'path forward',
        r'adoption strategy',
        r'deployment',
        r'rollout'
    ],
    'risk_assessment': [
        r'risks',
        r'challenges',
        r'barriers',
        r'mitigation',
        r'considerations',
        r'risk factors',
        r'obstacles',
        r'hurdles'
    ],
    'recommendations': [
        r'recommendations',
        r'key actions',
        r'strategic priorities',
        r'calls to action',
        r'proposed actions',
        r'way forward'
    ],
    'financial_analysis': [
        r'financial analysis',
        r'revenue potential',
        r'profitability',
        r'margin impact',
        r'financial model',
        r'economic model'
    ],
    'organizational_impact': [
        r'organizational impact',
        r'talent implications',
        r'workforce',
        r'capabilities',
        r'skills',
        r'operational impact'
    ]
}

# BCG-Specific Patterns (Consulting & Research Focus)
bcg_specific_patterns = {
    'research_findings': [
        r'research findings',
        r'study results', 
        r'empirical proof',
        r'survey results',
        r'data shows',
        r'our research'
    ],
    'exhibits_data': [
        r'exhibit \d+',
        r'figure \d+',
        r'table \d+',
        r'data exhibit',
        r'research data',
        r'self-reported impact'
    ],
    'value_gap_analysis': [
        r'value gap',
        r'future-built',
        r'laggards',
        r'stagnating',
        r'emerging',
        r'scaling',
        r'maturity curve',
        r'virtuous cycle',
        r'vicious cycle'
    ],
    'case_studies_examples': [
        r'for example',
        r'case in point',
        r'consider the case',
        r'global leader in',
        r'major.*company',
        r'leading.*firm'
    ],
    'implementation_framework': [
        r'playbook',
        r'framework',
        r'10-20-70',
        r'operating model',
        r'strategic priorities',
        r'roadmap',
        r'maturity score'
    ],
    'ai_definitions': [
        r'ai definitions',
        r'methodology',
        r'survey methodology',
        r'maturity categories',
        r'value realization pathways'
    ],
    'sector_analysis': [
        r'by sector',
        r'by region', 
        r'industry analysis',
        r'maturity varies',
        r'regional differences'
    ]
}

# Main function to get appropriate patterns

def get_section_patterns(report_source, content_preview=""):
    source_lower = report_source.lower()
    content_preview_lower = content_preview.lower()
    
    # Detect report type
    if any(keyword in source_lower for keyword in ['weforum', 'world economic forum', 'emerging technologies']):
        return {**universal_patterns, **wef_specific_patterns}
    elif any(keyword in source_lower for keyword in ['mckinsey', 'company', 'business review']):
        return {**universal_patterns, **mckinsey_specific_patterns}
    elif any(keyword in source_lower for keyword in ['bcg', 'boston consulting']):
        return {**universal_patterns, **bcg_specific_patterns}
    
    # Content-based fallback detection
    elif any(keyword in content_preview_lower for keyword in ['technology', 'emerging', 'innovation', 'future']):
        return {**universal_patterns, **wef_specific_patterns}
    elif any(keyword in content_preview_lower for keyword in ['business', 'roi', 'market', 'strategy', 'growth']):
        return {**universal_patterns, **mckinsey_specific_patterns}
    elif any(keyword in content_preview_lower for keyword in ['value gap', 'future-built', 'exhibit', 'research findings']):
        return {**universal_patterns, **bcg_specific_patterns}
    else:
        return universal_patterns  # fallback to universal only

# Extracting key sections

def extract_key_sections(text, report_name):
    """Extract important sections from reports"""
    # Get appropriate patterns based on report type
    section_patterns = get_section_patterns(report_name, text[:1000])  # Use first 1000 chars for detection
    
    sections = {
        'executive_summary': '',
        'key_findings': '',
        'main_conclusions': '',
        'methodology': '',
        'introduction': '',
        'technology_catalog': '',
        'strategic_outlook': '',
        'business_case': '',
        'recommendations': ''
    }
    
    lines = text.split('\n')
    current_section = 'introduction'
    section_content = []
    
    for line in lines:
        line_lower = line.lower().strip()
        
        # Check if this line starts a new section
        section_found = False
        for section_name, patterns in section_patterns.items():
            for pattern in patterns:
                if re.search(pattern, line_lower) and len(line) < 100:  # Avoid matching body text
                    if section_content:
                        sections[current_section] = '\n'.join(section_content)
                    current_section = section_name
                    section_content = [line]  # Include the section header
                    section_found = True
                    break
            if section_found:
                break
        
        if not section_found:
            section_content.append(line)
    
    # Add the last section
    if section_content:
        sections[current_section] = '\n'.join(section_content)
    
    return sections

# Function for processing one report

def process_report(pdf_path, output_path):
    """Process a single technology report"""
    
    pdf_file = Path(pdf_path)
    if not pdf_file.exists():
        print(f"❌ PDF file not found: {pdf_file.absolute()}")
        return False
    
    print(f"📄 Processing: {pdf_file.name}")
    
    try:
        # Extract text from PDF
        raw_text = extract_pdf_text(pdf_path)
        if not raw_text:
            print(f"❌ Could not extract text from {pdf_file.name}")
            return False
        
        print(f"   Extracted {len(raw_text)} characters")
        
        # Clean the text
        clean_text = clean_report_text(raw_text)
        
        # Extract key sections
        sections = extract_key_sections(clean_text, pdf_file.name)
        
        # Create output directory
        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Save processed report
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(f"TECH REPORT: {pdf_file.stem}\n")
            f.write(f"SOURCE: {pdf_file.name}\n")
            f.write("="*60 + "\n\n")
            
            # Write key sections first
            f.write("## KEY SECTIONS ##\n\n")
            for section_name, section_content in sections.items():
                if section_content and len(section_content) > 50:  # Only include substantial sections
                    f.write(f"### {section_name.upper().replace('_', ' ')} ###\n")
                    f.write(section_content)
                    f.write("\n\n" + "-"*50 + "\n\n")
            
            # Write full content
            f.write("## FULL REPORT CONTENT ##\n\n")
            f.write(clean_text)
        
        print(f"✅ Saved: {output_path}")
        return True
        
    except Exception as e:
        print(f"❌ Error processing {pdf_file.name}: {e}")
        return False
    
# Function for processing all tech reports

def process_all_tech_reports():
    """Process all three technology reports"""
    
    reports = [
        {
            'name': 'WEF Emerging Technologies 2025',
            'input': '../../01_data/rag_automotive_tech/raw_sources/tech_reports/wef_emerging_technologies_2025.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/tech_reports/wef_emerging_tech_2025.txt'
        },
        {
            'name': 'McKinsey Technology Trends 2025', 
            'input': '../../01_data/rag_automotive_tech/raw_sources/tech_reports/mckinsey_tech_trends_2025.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/tech_reports/mckinsey_tech_trends_2025.txt'
        },
        {
            'name': 'BCG AI Value 2025',
            'input': '../../01_data/rag_automotive_tech/raw_sources/tech_reports/bcg_ai_value_2025.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/tech_reports/bcg_ai_value_2025.txt'
        }
    ]
    
    print("🚀 PROCESSING TECH REPORTS")
    print("=" * 50)
    
    success_count = 0
    for report in reports:
        success = process_report(report['input'], report['output'])
        if success:
            success_count += 1
        print()  # Empty line between reports
    
    print(f"📊 Completed: {success_count}/{len(reports)} reports processed successfully")
    return success_count == len(reports)

# Run the processing for tech reports
if __name__ == "__main__":
    success = process_all_tech_reports()
    if success:
        print("\n🎉 All tech reports processed successfully!")
    else:
        print("\n⚠️ Some reports failed to process. Check the errors above.")

🚀 PROCESSING TECH REPORTS
📄 Processing: wef_emerging_technologies_2025.pdf
   Extracted 143886 characters
✅ Saved: ../../01_data/rag_automotive_tech/processed/tech_reports/wef_emerging_tech_2025.txt

📄 Processing: mckinsey_tech_trends_2025.pdf
   Extracted 310965 characters
✅ Saved: ../../01_data/rag_automotive_tech/processed/tech_reports/mckinsey_tech_trends_2025.txt

📄 Processing: bcg_ai_value_2025.pdf
   Extracted 64762 characters
✅ Saved: ../../01_data/rag_automotive_tech/processed/tech_reports/bcg_ai_value_2025.txt

📊 Completed: 3/3 reports processed successfully

🎉 All tech reports processed successfully!


#### Automative Papers PDFs

In [6]:
# Function for processing all automative reports

def process_all_automative_reports():
    """Process all three technology reports"""
    
    reports = [
        {
            'name': 'A Benchmark Framework for AI models in automative aerodynamics',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/a_benchmark_framework_for_AI_models_in_automative_aerodynamics.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/a_benchmark_framework_for_AI_models_in_automative_aerodynamics.txt'
        },
        {
            'name': 'AI agents in engineering design a multiagent framework for Aesthetic and Aerodynamic car design',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/AI_agents_in_engineering_design_a_multiagent_framework_for_aesthetic_and_aerodynamic_car_design.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/AI_agents_in_engineering_design_a_multiagent_framework_for_aesthetic_and_aerodynamic_car_design.txt'
        },
        {
            'name': 'Automating Automative Software Development: A synergy of generative AI and formal methods',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/automating_automative_software_development_a_synergy_of_generative_AI_and_formal_methods.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/automating_automative_software_development_a_synergy_of_generative_AI_and_formal_methods.txt'
        },
        {
            'name': 'Automotive Software and Electronics 2030 full report',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/automotive-software-and-electronics-2030-full-report.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/automotive-software-and-electronics-2030-full-report.txt'
        },
        {
            'name': 'Drive Disfluency-rich Synthetic Dialog data generation framework for intelligent vehicle environments',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/drive_disfluency-rich_synthetic_dialog_data_generation_framework_for_intelligent_vehicle_environments.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/drive_disfluency-rich_synthetic_dialog_data_generation_framework_for_intelligent_vehicle_environments.txt'
        },
        {
            'name': 'Embedded acoustic intelligence for Automotive Systems',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/Embedded_acoustic_intelligence_for_automotive_systems.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/Embedded_acoustic_intelligence_for_automotive_systems.txt'
        },
        {
            'name': 'Enhanced drift aware computer vision achitecture for autonomous driving',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/enhanced_drift_aware_computer_vision_achitecture_for_autonomous_driving.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/enhanced_drift_aware_computer_vision_achitecture_for_autonomous_driving.txt'
        },
        {
            'name': 'Gen AI in Automotive Applications: Challenges and opportunities with a case study on in-vehicle experience',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/Gen_AI_in_automotive_applications_challenges_and_opportunities_with_a_case_study_on_in-vehicle_experience.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/Gen_AI_in_automotive_applications_challenges_and_opportunities_with_a_case_study_on_in-vehicle_experience.txt'
        },
        {
            'name': 'Generative AI for autonomous driving: A review',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/generative_AI_for_autonomous_driving_a_review.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/generative_AI_for_autonomous_driving_a_review.txt'
        },
        {
            'name': 'Leveraging vision language models for visual grounding and analysis of automative UI',
            'input': '../../01_data/rag_automotive_tech/raw_sources/automotive_papers/leveraging_vision_language_models_for_visual_grounding_and_analysis_of_automative_UI.pdf',
            'output': '../../01_data/rag_automotive_tech/processed/automotive_papers/leveraging_vision_language_models_for_visual_grounding_and_analysis_of_automative_UI.txt'
        }
    ]
    
    print("🚀 PROCESSING AUTOMOTIVE REPORTS")
    print("=" * 50)
    
    success_count = 0
    for report in reports:
        success = process_report(report['input'], report['output'])
        if success:
            success_count += 1
        print()  # Empty line between reports
    
    print(f"📊 Completed: {success_count}/{len(reports)} reports processed successfully")
    return success_count == len(reports)

# Run the processing for automative reports
if __name__ == "__main__":
    success = process_all_automative_reports()
    if success:
        print("\n🎉 All automative reports processed successfully!")
    else:
        print("\n⚠️ Some reports failed to process. Check the errors above.")

🚀 PROCESSING AUTOMOTIVE REPORTS
📄 Processing: a_benchmark_framework_for_AI_models_in_automative_aerodynamics.pdf
   Extracted 50659 characters
✅ Saved: ../../01_data/rag_automotive_tech/processed/automotive_papers/a_benchmark_framework_for_AI_models_in_automative_aerodynamics.txt

📄 Processing: AI_agents_in_engineering_design_a_multiagent_framework_for_aesthetic_and_aerodynamic_car_design.pdf
   Extracted 71766 characters
✅ Saved: ../../01_data/rag_automotive_tech/processed/automotive_papers/AI_agents_in_engineering_design_a_multiagent_framework_for_aesthetic_and_aerodynamic_car_design.txt

📄 Processing: automating_automative_software_development_a_synergy_of_generative_AI_and_formal_methods.pdf
   Extracted 43844 characters
✅ Saved: ../../01_data/rag_automotive_tech/processed/automotive_papers/automating_automative_software_development_a_synergy_of_generative_AI_and_formal_methods.txt

📄 Processing: automotive-software-and-electronics-2030-full-report.pdf
   Extracted 92001 characters